<a href="https://colab.research.google.com/github/local-ai-gr/CNROpt/blob/main/CNROpt_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CNROpt Model Training Code for the Forecasting Phase**

Importing Facebook Prophet Forecasting Module

In [ ]:
!pip install prophet



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!pip install functions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for functions: filename=functions-0.7.0-py3-none-any.whl size=3037 sha256=e3542bb5906022ae7d22e2850fee99e8b6729729ae03b4121419ba724b5ff105
  Stored in directory: /root/.cache/pip/wheels/92/90/bc/01eb2f163bcfa49c7ad5072fbe9d243038368b9001ec26a9da
Successfully built functions


In [11]:
import pandas as pd
import prophet as ph
import psycopg2
import os
import sys
from functions import sql_to_dataframe

SyntaxError: ignored

Connecting to PostGIS Database (host: 34.171.207.247:5432
db: localai) and loading data (all but the test set) to pandas Data Frame

In [4]:
conn = psycopg2.connect(
                   host='34.171.207.247',
                   database='localai',
                   user='postgres',
                   password='pglocalai')

Run Prophet and Test it using various settings, and on the various KPIs such as 

Report the results for each configuration.



In [16]:
query = "select * from station_data_hour"
cursor = conn.cursor()
cursor.execute(query)
tupples = cursor.fetchall()
cursor.close()

df = pd.DataFrame(tupples)


Visualize the results using matplotlib and folium maps

In [17]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,107579,Koprivnica Mihanovićeva,HRHEP-48001,Koprivnica,48000,Ulica Antuna Mihanovića,9,Public access - Free access,2019-09-15 12:00:00+00:00,5.000000,7,0,2.477495
1,107579,Koprivnica Mihanovićeva,HRHEP-48001,Koprivnica,48000,Ulica Antuna Mihanovića,9,Public access - Free access,2019-09-15 13:00:00+00:00,14.285714,7,1,7.078557
2,107579,Koprivnica Mihanovićeva,HRHEP-48001,Koprivnica,48000,Ulica Antuna Mihanovića,9,Public access - Free access,2019-09-15 14:00:00+00:00,13.095238,7,0,6.488678
3,107579,Koprivnica Mihanovićeva,HRHEP-48001,Koprivnica,48000,Ulica Antuna Mihanovića,9,Public access - Free access,2019-09-16 07:00:00+00:00,1.428571,7,0,1.778800
4,107579,Koprivnica Mihanovićeva,HRHEP-48001,Koprivnica,48000,Ulica Antuna Mihanovića,9,Public access - Free access,2019-09-16 08:00:00+00:00,2.142857,7,0,2.668201
...,...,...,...,...,...,...,...,...,...,...,...,...,...
435210,359766,Rijeka Jolly,HRHEP-51030,Rijeka,51118,Osječka,39/A,Public access - Free access,2022-10-13 13:00:00+00:00,45.833333,2,0,4.453712
435211,359766,Rijeka Jolly,HRHEP-51030,Rijeka,51118,Osječka,39/A,Public access - Free access,2022-10-21 11:00:00+00:00,39.166667,2,0,5.640000
435212,359766,Rijeka Jolly,HRHEP-51030,Rijeka,51118,Osječka,39/A,Public access - Free access,2022-10-21 12:00:00+00:00,50.000000,2,1,7.200000
435213,359766,Rijeka Jolly,HRHEP-51030,Rijeka,51118,Osječka,39/A,Public access - Free access,2022-10-21 13:00:00+00:00,14.166667,2,0,2.040000
